# RAG Application with Automated Document Parser

This notebook demonstrates how to build a complete RAG (Retrieval-Augmented Generation) application using the automated-document-parser package and LangChain with HuggingFace models.

RAG enables you to:
- Load and process documents from various formats
- Create embeddings and store them in a vector database
- Retrieve relevant context for user queries
- Generate accurate answers based on your documents

## 1. Import Required Libraries

Import all necessary components for our RAG application.

In [1]:
import os
from pathlib import Path

# Our custom document parser
from automated_document_parser import DocumentParser

# LangChain components
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

print("All libraries imported successfully!")

/Users/pulkit/Desktop/test/document_parser/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries imported successfully!


## 3. Configure HuggingFace Models

We'll use free, small HuggingFace models for testing. No API key required!

In [2]:
# Configure models to use
# We'll use small, efficient models for testing

# Embedding model - small and fast
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # 384 dimensions, 22M parameters

# Language model for generation - small model for testing
LLM_MODEL = "google/flan-t5-small"  # 60M parameters, good for testing

print(f"Embedding model: {EMBEDDING_MODEL}")
print(f"Language model: {LLM_MODEL}")
print("No API key required - models run locally!")

Embedding model: sentence-transformers/all-MiniLM-L6-v2
Language model: google/flan-t5-small
No API key required - models run locally!


## 4. Create Sample Documents

Let's create some sample documents for our RAG system to demonstrate the functionality.

In [3]:
# Create sample documents directory
sample_docs_dir = Path("sample_docs")
sample_docs_dir.mkdir(exist_ok=True)

# Create sample text file
with open(sample_docs_dir / "python_basics.txt", "w") as f:
    f.write("""
Python Programming Basics

Python is a high-level, interpreted programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

Key Features:
- Easy to learn and read
- Dynamically typed
- Object-oriented
- Large standard library
- Cross-platform compatibility

Common Use Cases:
- Web development (Django, Flask)
- Data science and machine learning
- Automation and scripting
- API development
""")

# Create another sample file
with open(sample_docs_dir / "machine_learning.txt", "w") as f:
    f.write("""
Introduction to Machine Learning

Machine Learning is a subset of artificial intelligence that enables systems to learn
and improve from experience without being explicitly programmed.

Types of Machine Learning:
1. Supervised Learning - Learning from labeled data
2. Unsupervised Learning - Finding patterns in unlabeled data
3. Reinforcement Learning - Learning through trial and error

Popular ML Libraries:
- Scikit-learn for classical ML
- TensorFlow and PyTorch for deep learning
- Pandas for data manipulation
- NumPy for numerical computations
""")

# Create a CSV file
with open(sample_docs_dir / "data.csv", "w") as f:
    f.write("""language,year_created,paradigm
Python,1991,Multi-paradigm
JavaScript,1995,Multi-paradigm
Java,1995,Object-oriented
Rust,2010,Multi-paradigm
""")

print(f"Sample documents created in {sample_docs_dir}/")
print(f"   - python_basics.txt")
print(f"   - machine_learning.txt")
print(f"   - data.csv")

Sample documents created in sample_docs/
   - python_basics.txt
   - machine_learning.txt
   - data.csv


## 5. Load Documents Using Automated Document Parser

Now we'll use DocumentParser with automatic file type detection:

**Step 1:** Specify the loading method (optional - uses defaults if not specified)
**Step 2:** Parser automatically detects file types and loads them with the specified settings

The parser intelligently detects file types based on extensions and uses the appropriate loader for each file.

In [ ]:
# Initialize the DocumentParser
parser = DocumentParser()

# Get all files in the sample directory
file_paths = [str(f) for f in sample_docs_dir.glob("*") if f.is_file()]

# Step 1: Specify the method (optional - defaults shown here)
# Step 2: Parser automatically detects each file type and loads appropriately
parsed_docs = parser.parse_multiple(
    file_paths,
    pdf_loader_method="pypdf",  # Method for PDF files (if any)
    encoding="utf-8"             # Encoding for text/CSV files
)

# Display loaded documents
total_docs = sum(len(docs) for docs in parsed_docs.values())
print(f"Loaded {total_docs} documents from {len(parsed_docs)} files:")
for file_path, docs in parsed_docs.items():
    print(f"  - {Path(file_path).name}: {len(docs)} document(s)")

# Flatten all documents into a single list
all_documents = []
for docs in parsed_docs.values():
    all_documents.extend(docs)

print(f"\nTotal document chunks: {len(all_documents)}")

# Display first document as example
if all_documents:
    print("\n--- Sample Document ---")
    print(f"Content preview: {all_documents[0].page_content[:200]}...")
    print(f"Metadata: {all_documents[0].metadata}")

Loaded 6 documents from 3 files:
  - python_basics.txt: 1 document(s)
  - data.csv: 4 document(s)
  - machine_learning.txt: 1 document(s)

Total document chunks: 6

--- Sample Document ---
Content preview: 
Python Programming Basics

Python is a high-level, interpreted programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

Key Featu...
Metadata: {'source': '/Users/pulkit/Desktop/test/document_parser/notebook_examples/sample_docs/python_basics.txt', 'file_name': 'python_basics.txt', 'file_type': '.txt'}


## 6. Split Documents into Chunks

For better retrieval accuracy, we split documents into smaller chunks. This ensures:
- Each chunk fits within the embedding model's context window
- More precise retrieval of relevant information
- Better matching between queries and document segments

In [5]:
# Split documents into smaller chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# Split all documents
split_documents = text_splitter.split_documents(all_documents)

print(f"Split {len(all_documents)} documents into {len(split_documents)} chunks")
print(f"\nSample chunk:")
print(f"Content: {split_documents[0].page_content[:200]}...")
print(f"Metadata: {split_documents[0].metadata}")

Split 6 documents into 7 chunks

Sample chunk:
Content: Python Programming Basics

Python is a high-level, interpreted programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

Key Featur...
Metadata: {'source': '/Users/pulkit/Desktop/test/document_parser/notebook_examples/sample_docs/python_basics.txt', 'file_name': 'python_basics.txt', 'file_type': '.txt'}


## 7. Create Embeddings and Vector Store

Initialize HuggingFace embeddings and create a FAISS vector store. The vector store enables:
- Fast similarity search across thousands of documents
- Efficient retrieval of relevant context
- Semantic matching beyond keyword search

In [6]:
# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print(f"Using embedding model: {EMBEDDING_MODEL}")
print("Creating vector store...")

# Create FAISS vector store
vectorstore = FAISS.from_documents(
    documents=split_documents,
    embedding=embeddings
)

print(f"Vector store created with {vectorstore.index.ntotal} embeddings")

# Test similarity search
query = "What is Python?"
results = vectorstore.similarity_search(query, k=2)
print(f"\nTest search for: '{query}'")
print(f"Found {len(results)} relevant chunks")
print(f"\nTop result preview:")
print(results[0].page_content[:200])

Using embedding model: sentence-transformers/all-MiniLM-L6-v2
Creating vector store...
Vector store created with 7 embeddings

Test search for: 'What is Python?'
Found 2 relevant chunks

Top result preview:
Python Programming Basics

Python is a high-level, interpreted programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

Key Featur
Vector store created with 7 embeddings

Test search for: 'What is Python?'
Found 2 relevant chunks

Top result preview:
Python Programming Basics

Python is a high-level, interpreted programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

Key Featur


## 8. Configure Language Model

Set up the HuggingFace language model for generating answers. We're using flan-t5-small, a compact model that's perfect for testing and learning.

In [7]:
# Initialize the language model pipeline
from transformers import pipeline

print(f"Loading language model: {LLM_MODEL}")
print("This may take a moment on first run...")

text_generation_pipeline = pipeline(
    "text2text-generation",
    model=LLM_MODEL,
    max_length=512,
    device=-1  # Use CPU
)

# Create LangChain LLM wrapper
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

print("Language model loaded successfully")

Loading language model: google/flan-t5-small
This may take a moment on first run...


Device set to use cpu


Language model loaded successfully


## 9. Create Simple Q&amp;A Function

Now we'll create a simple question-answering function that:
1. **Retrieves**: Searches the vector store for relevant documents
2. **Generates**: Uses the LLM to answer based on retrieved context
3. **Returns**: Provides a single answer without maintaining history

In [8]:
# Create a simple Q&amp;A function
def answer_question(question, retriever, llm, k=3):
    """
    Answer a question based on retrieved documents.
    
    Args:
        question: The user's question
        retriever: Vector store retriever
        llm: Language model
        k: Number of documents to retrieve
    
    Returns:
        tuple: (answer, source_documents)
    """
    # Retrieve relevant documents
    docs = retriever.invoke(question)
    
    # Combine document contents
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Create prompt
    prompt = f"""Answer the question based on the context below. If the answer cannot be found in the context, say "I don't have enough information to answer that."

Context:
{context}

Question: {question}

Answer:"""
    
    # Generate answer
    answer = llm.invoke(prompt)
    
    return answer, docs

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print("Simple Q&amp;A function created successfully!")
print("\nReady to answer questions based on your documents!")


Simple Q&amp;A function created successfully!

Ready to answer questions based on your documents!


## 10. Ask Questions and Get Answers

Now let's test our Q&amp;A function! The system will:
1. Take your question
2. Search for relevant document chunks
3. Generate an answer based on the context
4. Return the answer with source attribution

In [9]:
# Ask questions about the documents
questions = [
    "What are the types of machine learning?",
    "Which programming language was created in 2010?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    print("-" * 80)
    
    answer, source_docs = answer_question(question, retriever, llm)
    
    print(f"Answer: {answer}")
    print(f"\nSources used:")
    for i, doc in enumerate(source_docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        file_name = Path(source).name if source != 'Unknown' else 'Unknown'
        print(f"  {i}. {file_name}")


Question: What are the types of machine learning?
--------------------------------------------------------------------------------
Answer: a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed

Sources used:
  1. machine_learning.txt
  2. machine_learning.txt
  3. data.csv

Question: Which programming language was created in 2010?
--------------------------------------------------------------------------------
Answer: Python

Sources used:
  1. data.csv
  2. data.csv
  3. data.csv
Answer: a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed

Sources used:
  1. machine_learning.txt
  2. machine_learning.txt
  3. data.csv

Question: Which programming language was created in 2010?
--------------------------------------------------------------------------------
Answer: Python

Sources used:
  1. data.csv
  2. data.csv
  3. data.csv


## 11. Advanced: Using Specific File Loaders

The document parser now has a modular structure with dedicated loaders for each file type. You can use specific loaders directly for more control:

### File Loaders (in `file_load/` module):
- **TextFileLoader** - For .txt and .md files
- **CSVFileLoader** - For CSV files with encoding support
- **JSONFileLoader** - For JSON files with jq schema support
- **DOCXFileLoader** - For Microsoft Word documents
- **HTMLFileLoader** - For HTML files

### PDF Loaders (in `pdf_load/` module):
- **pypdf** - Basic PDF text extraction
- **unstructured** - Advanced OCR and layout detection
- **amazon_textract** - AWS Textract for high-accuracy OCR
- **mathpix** - Specialized for mathematical formulas
- **pdfplumber** - High accuracy text and table extraction
- **pypdfium2** - Google PDFium library
- **pymupdf** - PyMuPDF (fitz) backend
- **pymupdf4llm** - LLM-optimized extraction
- **opendataloader** - Advanced multi-format parsing

In [10]:
# Example: Using specific file loaders directly
from automated_document_parser.loaders.file_load import (
    TextFileLoader,
    CSVFileLoader,
    JSONFileLoader
)

# Load a text file with specific encoding
text_loader = TextFileLoader(
    file_path=str(sample_docs_dir / "python_basics.txt"),
    encoding="utf-8"
)
text_docs = text_loader.load()
print(f"Loaded {len(text_docs)} document(s) from text file")
print(f"Preview: {text_docs[0].page_content[:150]}...\n")

# Load a CSV file
csv_loader = CSVFileLoader(
    file_path=str(sample_docs_dir / "data.csv"),
    encoding="utf-8"
)
csv_docs = csv_loader.load()
print(f"Loaded {len(csv_docs)} row(s) from CSV file")
print(f"First row: {csv_docs[0].page_content[:100]}...\n")

# Get installation commands if needed
print("Installation commands for loaders:")
print(f"  Text: {text_loader.get_install_command()}")
print(f"  CSV: {csv_loader.get_install_command()}")

Loaded 1 document(s) from text file
Preview: 
Python Programming Basics

Python is a high-level, interpreted programming language known for its simplicity and readability.
It was created by Guido...

Loaded 4 row(s) from CSV file
First row: language: Python
year_created: 1991
paradigm: Multi-paradigm...

Installation commands for loaders:
  Text: pip install langchain-community
  CSV: pip install langchain-community


## 12. Advanced: Using Different PDF Loaders

The document parser supports multiple PDF loading methods, each optimized for different use cases. Let's explore how to use them:

**Note:** Most PDF loaders require additional dependencies. You can install them as needed.

In [11]:
# Example: Using different PDF loading methods
from automated_document_parser.loaders import PDFLoader, load_pdf

# Note: For this demo, we'll show the API without actually loading PDFs
# since we don't have sample PDF files in this notebook

# Method 1: Using PyPDF (default - no extra dependencies)
print("1. PyPDF Loader (default)")
print("   - Basic PDF text extraction")
print("   - No extra dependencies required")
print("   Usage: PDFLoader('file.pdf', method='pypdf')\n")

# Method 2: Using Mathpix for mathematical content
print("2. Mathpix Loader")
print("   - Specialized for mathematical formulas and equations")
print("   - Requires: langchain-community")
print("   - Requires API key: set MATHPIX_APP_ID and MATHPIX_APP_KEY")
print("   Usage: PDFLoader('file.pdf', method='mathpix', mathpix_app_id='...', mathpix_app_key='...')\n")

# Method 3: Using PDFPlumber for high accuracy
print("3. PDFPlumber Loader")
print("   - High accuracy text and table extraction")
print("   - Requires: pdfplumber")
print("   - Install: pip install pdfplumber")
print("   Usage: PDFLoader('file.pdf', method='pdfplumber')\n")

# Method 4: Using PyMuPDF for fast processing
print("4. PyMuPDF Loader")
print("   - Fast PDF processing with fitz backend")
print("   - Requires: pymupdf")
print("   - Install: pip install pymupdf")
print("   Usage: PDFLoader('file.pdf', method='pymupdf')\n")

# Method 5: Using PyMuPDF4LLM for LLM-optimized extraction
print("5. PyMuPDF4LLM Loader")
print("   - Optimized for LLM consumption")
print("   - Preserves document structure for better context")
print("   - Requires: pymupdf4llm")
print("   - Install: pip install pymupdf4llm")
print("   Usage: PDFLoader('file.pdf', method='pymupdf4llm')\n")

# Get installation command for any loader
print("\nTo check installation requirements:")
print("loader = PDFLoader('file.pdf', method='pdfplumber')")
print("print(loader.get_install_command())")

1. PyPDF Loader (default)
   - Basic PDF text extraction
   - No extra dependencies required
   Usage: PDFLoader('file.pdf', method='pypdf')

2. Mathpix Loader
   - Specialized for mathematical formulas and equations
   - Requires: langchain-community
   - Requires API key: set MATHPIX_APP_ID and MATHPIX_APP_KEY
   Usage: PDFLoader('file.pdf', method='mathpix', mathpix_app_id='...', mathpix_app_key='...')

3. PDFPlumber Loader
   - High accuracy text and table extraction
   - Requires: pdfplumber
   - Install: pip install pdfplumber
   Usage: PDFLoader('file.pdf', method='pdfplumber')

4. PyMuPDF Loader
   - Fast PDF processing with fitz backend
   - Requires: pymupdf
   - Install: pip install pymupdf
   Usage: PDFLoader('file.pdf', method='pymupdf')

5. PyMuPDF4LLM Loader
   - Optimized for LLM consumption
   - Preserves document structure for better context
   - Requires: pymupdf4llm
   - Install: pip install pymupdf4llm
   Usage: PDFLoader('file.pdf', method='pymupdf4llm')


To chec

## 13. Practical Example: Choosing the Right Loader

Here's a guide to help you choose the right loader for your use case:

### For Text Files (.txt, .md):
- Use `TextFileLoader` for simple text files
- Supports custom encoding (UTF-8, Latin-1, etc.)

### For Structured Data:
- **CSV**: Use `CSVFileLoader` for tabular data
- **JSON**: Use `JSONFileLoader` with optional jq schema filtering

### For Documents:
- **DOCX**: Use `DOCXFileLoader` for Microsoft Word documents
- **HTML**: Use `HTMLFileLoader` for web pages

### For PDFs:
- **General text**: Use `pypdf` (default, no dependencies)
- **Mathematical content**: Use `mathpix` (requires API key)
- **High accuracy tables**: Use `pdfplumber`
- **Fast processing**: Use `pymupdf`
- **LLM-optimized**: Use `pymupdf4llm`
- **Complex layouts**: Use `unstructured` or `amazon_textract`

### Auto-detection:
For most cases, just use `DocumentParser()` - it automatically selects the right loader based on file extension!

In [ ]:
# Complete example: Mixed document types with auto-detection
from automated_document_parser import DocumentParser

# Initialize parser
parser = DocumentParser()

# Let's see what files we have
print("Available files in sample_docs:")
for file in sample_docs_dir.glob("*"):
    if file.is_file():
        print(f"  - {file.name} ({file.suffix})")

# Parse all files automatically - parser detects file types
print("\nParsing all files with auto-detection...")
all_files = [str(f) for f in sample_docs_dir.glob("*") if f.is_file()]
results = parser.parse_multiple(all_files)

print(f"\nSuccessfully parsed {len(results)} files")
for file_path, docs in results.items():
    file_name = Path(file_path).name
    file_type = Path(file_path).suffix
    print(f"  {file_name} ({file_type}): {len(docs)} document(s)")

# Show loaded files
print(f"\nTotal files loaded: {len(parser.get_loaded_files())}")
print("Parser automatically selected the right loader for each file type!")

Available files in sample_docs:
  - python_basics.txt (.txt)
  - data.csv (.csv)
  - machine_learning.txt (.txt)

Parsing all files with auto-detection...

✓ Successfully parsed 3 files
  python_basics.txt (.txt): 1 document(s)
  data.csv (.csv): 4 document(s)
  machine_learning.txt (.txt): 1 document(s)

Total files loaded: 3
Parser automatically selected the right loader for each file type!


## Summary

Congratulations! You've learned how to:

1. Set up a complete RAG application with HuggingFace models
2. Use the automated document parser to load various file types
3. Create embeddings and build a vector store
4. Implement question-answering with context retrieval
5. Use specific file loaders for granular control
6. Choose the right PDF loader for your use case

### Key Takeaways:

- **Modular Structure**: The parser now has `file_load/` and `pdf_load/` subdirectories for organized loaders
- **Auto-Detection**: Use `DocumentParser()` for automatic file type detection
- **Flexibility**: Import specific loaders when you need fine-grained control
- **9 PDF Methods**: Choose from multiple PDF loading strategies based on your needs
- **No API Keys Required**: This demo uses free HuggingFace models that run locally

### Next Steps:

1. Try adding your own documents to the `sample_docs/` folder
2. Experiment with different PDF loading methods
3. Use larger, more powerful models for production
4. Integrate with your own applications

Happy coding!